### LIBRARY IMPORT

In [38]:
import numpy as np
import pandas as pd

from contextlib import contextmanager
import time  
from sklearn.neighbors import NearestNeighbors  
from sklearn.preprocessing import minmax_scale  
from typing import Dict, List, Optional, Tuple  
import seaborn as sns 
import gc
import traceback 

import matplotlib.pyplot as plt

from scipy.stats import kendalltau

import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'  ## 서로 다른 환경에서는 이곳을 수정해야 함.
os.chdir(working_directory)



### CHECK NULL VALUE

In [39]:
# combined_result_df = pd.read_csv("./DB/professor_BTC_sum_both_10m.csv")

# SET the coin
coin = 'DOGE'

combined_result_df_raw = pd.read_csv("./output/{}_sum_both_10m.csv".format(coin))
print("# of rows of combined_Result_Df:", combined_result_df_raw.shape[0])

combined_result_df_raw['window_start'] = pd.to_datetime(combined_result_df_raw['window_start'])  # Convert to datetime

# Define the time range
start_time = pd.to_datetime('00:00:00').time()
end_time = pd.to_datetime('06:00:00').time()

# Filter and drop rows
filtered_df = combined_result_df_raw[~combined_result_df_raw['window_start'].apply(lambda x: start_time <= x.time() <= end_time)]
print("# of rows of filtered_df:", filtered_df.shape[0])

combined_result_df = filtered_df.dropna()
print("# of rows of filtered_df:", combined_result_df.shape[0])

# 평균이 아닌 이전 값으로 Null 채우기
# combined_result_df = combined_result_df.fillna(method='ffill') 


# of rows of combined_Result_Df: 100481
# of rows of filtered_df: 76330
# of rows of filtered_df: 75512


### Add Feature  

In [40]:
combined_result_df['liq_last_1'] = np.log10(combined_result_df['liq_last_1'] + 0.00001)
combined_result_df['liq_last_2'] = np.log10(combined_result_df['liq_last_2'] + 0.00001)
combined_result_df['liq_last_5'] = np.log10(combined_result_df['liq_last_5'] + 0.00001)
combined_result_df['liq_last_10'] = np.log10(combined_result_df['liq_last_10'] + 0.00001)
combined_result_df['liq_last_15'] = np.log10(combined_result_df['liq_last_15'] + 0.00001)
combined_result_df['trade_vol'] = np.log10(combined_result_df['trade_vol'] + 0.00001)
combined_result_df['num_trades'] = np.log10(combined_result_df['num_trades'] + 0.00001)

combined_result_df['trade.tau'] = np.sqrt(1 / combined_result_df['num_trades'])
combined_result_df['tvpl1'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_1']
combined_result_df['tvpl2'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_2'] 
combined_result_df['tvpl5'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_5'] 
combined_result_df['tvpl10'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_10'] 

c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

### NUMERIC FEATURES & CALCULATE CORR 

In [41]:
main_feature_list = list(combined_result_df.columns)
main_feature_list.remove('window_start')
main_feature_list.remove('window_end')
main_feature_list.remove('time_id')
main_feature_list.remove('volume_power')
main_feature_list.remove('dv1_realized_volatility')
main_feature_list.remove('dv2_lowest_return')
main_feature_list.remove('dv3_highest_return')
main_feature_list.remove('dv4_realized_volatility_30s')
main_feature_list.remove('prices_30s_for_NN')
main_feature_list.remove('window_end_150_ticker')
main_feature_list.remove('window_end_300_ticker')
main_feature_list.remove('window_end_450_ticker')
main_feature_list.remove('window_end_150_orderbook')
main_feature_list.remove('window_end_300_orderbook')
main_feature_list.remove('window_end_450_orderbook')
main_feature_list.remove('volume_power_150')
main_feature_list.remove('volume_power_300')


In [42]:
pd.set_option('display.max_columns', None)

combined_result_df.head(3)

,window_start,window_end,realized_volatility,num_trades,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,end_price,prices_30s_for_NN,time_id,BB_width_w20,BB_width_w40,BB_width_w10,dv1_realized_volatility,dv2_lowest_return,dv3_highest_return,window_end_150_ticker,realized_volatility_150,num_trades_150,lowest_return_150,highest_return_150,high_low_gap_150,trade_vol_150,volume_power_150,window_end_300_ticker,realized_volatility_300,num_trades_300,lowest_return_300,highest_return_300,high_low_gap_300,trade_vol_300,volume_power_300,window_end_450_ticker,realized_volatility_450,num_trades_450,lowest_return_450,highest_return_450,high_low_gap_450,trade_vol_450,volume_power_450,liq_last_1,liq_last_2,liq_last_5,liq_last_10,liq_last_15,ep_liq_5,bidask_spread_0,bidask_spread_1,window_end_150_orderbook,liq_last_1_150,liq_last_2_150,liq_last_5_150,liq_last_10_150,liq_last_15_150,bidask_spread_0_150,bidask_spread_1_150,window_end_300_orderbook,liq_last_1_300,liq_last_2_300,liq_last_5_300,liq_last_10_300,liq_last_15_300,bidask_spread_0_300,bidask_spread_1_300,window_end_450_orderbook,liq_last_1_450,liq_last_2_450,liq_last_5_450,liq_last_10_450,liq_last_15_450,bidask_spread_0_450,bidask_spread_1_450,realized_volatility_30s,dv4_realized_volatility_30s,tvpl,tvpl_epliq5,trade.tau,tvpl1,tvpl2,tvpl5,tvpl10
38,2022-12-16 21:44:00,2022-12-16 21:54:00,0.005318,2.064458,-0.009132,0.0,0.009132,5.582092,1.416817,109.0,datetime\n2022-12-16 21:44:00 0.000000\n202...,2022-12-16 21:44:00,1.777047,1.961161,1.264911,0.005320,0.000000,0.009132,2022-12-16 21:54:00,0.005023,24,-0.009132,0.0,0.009132,173256.710573,-1.222725,2022-12-16 21:54:00,0.005038,47,-0.009132,0.0,0.009132,236945.449831,1.340702,2022-12-16 21:54:00,0.005507,78,-0.009132,0.0,0.009132,287062.474955,1.338467,7.946419,8.056656,8.140713,8.204331,8.214321,1.373803e+08,10.000151,30.000460,2022-12-16 21:54:00,8.670641e+07,1.122187e+08,1.365022e+08,1.582729e+08,1.620013e+08,10.000153,30.000463,2022-12-16 21:54:00,8.826273e+07,1.136482e+08,1.379653e+08,1.597432e+08,1.634685e+08,10.00015,30.000462,2022-12-16 21:54:00,8.877480e+07,1.142026e+08,1.385255e+08,1.602915e+08,1.640160e+08,10.000153,30.000460,0.015818,0.035370,0.003353,0.002781,0.695980,0.702466,0.692855,0.685701,0.680384
39,2022-12-16 21:45:00,2022-12-16 21:55:00,0.005426,2.079181,-0.009132,0.0,0.009132,5.597348,1.444811,110.0,datetime\n2022-12-16 21:45:00 0.000000\n202...,2022-12-16 21:45:00,1.777047,1.932184,1.264911,0.005066,-0.009132,0.000000,2022-12-16 21:55:00,0.005273,25,-0.009132,0.0,0.009132,154278.147222,1.038443,2022-12-16 21:55:00,0.005479,51,-0.009132,0.0,0.009132,231369.144824,1.399868,2022-12-16 21:55:00,0.005524,83,-0.009132,0.0,0.009132,295028.249748,1.412203,7.934535,8.047794,8.133403,8.197882,8.208015,1.374485e+08,10.000150,30.000460,2022-12-16 21:55:00,8.760839e+07,1.131057e+08,1.373502e+08,1.591233e+08,1.628514e+08,10.000153,30.000463,2022-12-16 21:55:00,8.813524e+07,1.135259e+08,1.378690e+08,1.596358e+08,1.633603e+08,10.00015,30.000462,2022-12-16 21:55:00,8.834025e+07,1.138954e+08,1.382297e+08,1.599946e+08,1.637197e+08,10.000153,30.000460,0.018265,0.018265,0.003544,0.002879,0.693512,0.705441,0.695513,0.688193,0.682780
40,2022-12-16 21:46:00,2022-12-16 21:56:00,0.005674,2.060698,-0.009132,0.0,0.009132,5.580897,-1.165452,110.0,datetime\n2022-12-16 21:46:00 -0.009132\n202...,2022-12-16 21:46:00,1.641565,1.897367,1.264911,0.005009,-0.009132,0.000000,2022-12-16 21:56:00,0.005507,23,-0.009132,0.0,0.009132,142185.218209,-1.142089,2022-12-16 21:56:00,0.006119,50,-0.009132,0.0,0.009132,181899.066214,-1.129355,2022-12-16 21:56:00,0.005671,84,-0.009132,0.0,0.009132,256929.570920,1.082251,7.938746,8.050932,8.135962,8.200067,8.210154,1.375683e+08,10.000153,30.000458,2022-12-16 21:56:00,8.751957e+07,1.129845e+08,1.372243e+08,1.589991e+08,1.627250e+08,10.000153,30.000463,2022-12-16 21:56:00,8.843443e+07,1.138386e+08,1.381655e+08,1.599293e+08,1.636543e+08,10.00015,30.000460,2022-12-16 21:56:00,8.888432e+07,1.143602e+08,1.

In [43]:
combined_result_df.columns

Index(['window_start', 'window_end', 'realized_volatility', 'num_trades',
       'lowest_return', 'highest_return', 'high_low_gap', 'trade_vol',
       'volume_power', 'end_price', 'prices_30s_for_NN', 'time_id',
       'BB_width_w20', 'BB_width_w40', 'BB_width_w10',
       'dv1_realized_volatility', 'dv2_lowest_return', 'dv3_highest_return',
       'window_end_150_ticker', 'realized_volatility_150', 'num_trades_150',
       'lowest_return_150', 'highest_return_150', 'high_low_gap_150',
       'trade_vol_150', 'volume_power_150', 'window_end_300_ticker',
       'realized_volatility_300', 'num_trades_300', 'lowest_return_300',
       'highest_return_300', 'high_low_gap_300', 'trade_vol_300',
       'volume_power_300', 'window_end_450_ticker', 'realized_volatility_450',
       'num_trades_450', 'lowest_return_450', 'highest_return_450',
       'high_low_gap_450', 'trade_vol_450', 'volume_power_450', 'liq_last_1',
       'liq_last_2', 'liq_last_5', 'liq_last_10', 'liq_last_15', 'ep_liq_5'

In [44]:
# data = combined_result_df

# # Drop non-numeric columns or columns with missing values
# main_feature_list = data.select_dtypes(include=[np.number]).columns

# # Calculate correlation matrix using kendalltau method
# correlation_matrix = data[main_feature_list].corr(method=lambda x, y: kendalltau(x, y).correlation)

# # main_feature_list.remove('dv1_realized_volatility')

# new_index = main_feature_list.difference(['dv1_realized_volatility', 'dv2_lowest_resturn', 'dv3_highest_return', 'dv4_realized_volatility_30s'])
# main_feature_list = new_index


In [55]:
# Check correlation table (Which variable is more correlated with the main DV?)
main_feature_list2 = list(combined_result_df.columns)
data = combined_result_df
correlation_matrix = data[main_feature_list2].corr(method=lambda x, y: kendalltau(x, y).correlation)
pd.set_option('display.max_rows', None)
sorted_df = correlation_matrix.sort_values(by='dv1_realized_volatility', ascending=False)
sorted_df['dv1_realized_volatility']

dv1_realized_volatility        1.000000
realized_volatility            0.546897
realized_volatility_450        0.523334
realized_volatility_300        0.499681
liq_last_1                     0.483530
liq_last_2                     0.474937
liq_last_1_450                 0.473556
liq_last_1_300                 0.468436
liq_last_2_450                 0.467452
liq_last_1_150                 0.464963
liq_last_5                     0.463978
liq_last_2_300                 0.463295
liq_last_2_150                 0.460845
ep_liq_5                       0.457741
liq_last_5_450                 0.456876
liq_last_10                    0.454837
realized_volatility_150        0.454670
liq_last_5_300                 0.453166
liq_last_5_150                 0.451019
liq_last_15                    0.450902
high_low_gap_300               0.449497
liq_last_10_450                0.447963
liq_last_10_300                0.444393
liq_last_15_450                0.444131
high_low_gap_150               0.442522


In [56]:
tempa = combined_result_df[main_feature_list].isnull()

tempa.sum().sum()

0

In [57]:
print(tempa.sum().sum())

0


### BUILD NEIGHBORS

In [58]:
N_NEIGHBORS_MAX = 65 

class Neighbors:
    def __init__(self, 
                 name: str, 
                 pivot: pd.DataFrame, 
                 p: float, 
                 metric: str = 'minkowski', 
                 metric_params: object = None, 
                 exclude_self: bool = True,
                 ):
        self.name = name
        self.exclude_self = exclude_self
        self.p = p
        self.metric = metric

        nn = NearestNeighbors(
            n_neighbors=N_NEIGHBORS_MAX, 
            p=p, 
            metric=metric, 
            metric_params=metric_params
        )
        
        nn.fit(pivot)
        _, self.neighbors = nn.kneighbors(pivot, return_distance=True)

        self.columns = self.index = self.feature_values = self.feature_col = None

    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        raise NotImplementedError()

    def make_nn_feature(self, n=5, agg=np.mean) -> pd.DataFrame:
        assert self.feature_values is not None, "should call rearrange_feature_values beforehand"

        start = 1 if self.exclude_self else 0

        pivot_aggs = pd.DataFrame(
            agg(self.feature_values[start:n,:,0], axis=0), 
            columns=self.columns, 
            index=self.index
        )

        dst = pivot_aggs.reset_index() # unstack().
        # print("dst.shape:", dst.shape)
        new_column_names = ['time_id', f'{self.feature_col}_nn{n}_{self.name}_{agg.__name__}'] # 3개를 예측했는데 2개만 들어왔다??
        dst.columns = new_column_names 
        return dst
    

class TimeIdNeighbors(Neighbors):
    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        # feature_pivot = df.pivot(index='time_id', values=feature_col)
        # feature_pivot = feature_pivot.fillna(feature_pivot.mean())

        feature_df = df[['time_id', feature_col]]
        feature_df.set_index('time_id', inplace=True)
        feature_df = feature_df.fillna(feature_df.mean())

        feature_values = np.zeros((N_NEIGHBORS_MAX, feature_df.shape[0], 1))

        for i in range(N_NEIGHBORS_MAX):
            feature_values[i, :, 0] += feature_df.values[self.neighbors[:, i], 0]

        self.columns = list(feature_df.columns)
        self.index = list(feature_df.index)
        self.feature_values = feature_values
        self.feature_col = feature_col

    def __repr__(self) -> str:
        return f"time-id NN (name={self.name}, metric={self.metric}, p={self.p})"


### PROGRESS CHECK FUNCTION

In [59]:
@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}초')

def print_trace(name: str = ''):
    print(f'{name or "익명"}에서 에러가 발생했습니다.')
    print(traceback.format_exc())


### GET NN CLASS

In [64]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

USE_ONE_FEATURE_C = True
USE_ONE_FEATURE_M_1 = True
USE_ONE_FEATURE_M_2 = True

USE_TWO_FEATURES = True

USE_ALL_FEATURES = True
USE_SEVALRAL_FEATURES = True

# Top 5 Related Feature
top_5_high_feat = list(correlation_matrix['realized_volatility'].sort_values().keys())[:5]
top_5_low_feat = list(correlation_matrix['realized_volatility'].sort_values().keys())[-6:-1]


# Top 5 Absolute Related Feature

sorted_data = correlation_matrix['realized_volatility'].abs().sort_values(ascending=False)

top_5_high_abs_feat = list(sorted_data.head(6).keys())[1:]
top_5_low_abs_feat = list(sorted_data.tail(5).keys())

# time_id_neighbors List 
time_id_neighbors: List[Neighbors] = []

with timer('knn fit'):
    df_pv = combined_result_df.copy()
    # df_pv = df_pv.drop(['window_start', 'window_end','volume_power'], axis=1)
    df_pv = df_pv.drop(['window_start', 'window_end'], axis=1)
    
    ## Drop columns with NaN std.
    # Calculate standard deviation for each column
    std_deviation = df_pv.std()
    # Identify columns with NaN standard deviation
    columns_with_nan_std = std_deviation[std_deviation.isna()].index
    # Drop columns with NaN standard deviation from the DataFrame
    df_pv.drop(columns=columns_with_nan_std, inplace=True)
    main_feature_list = [item for item in main_feature_list if item not in columns_with_nan_std]

    top_5_high_abs_feat = [item for item in top_5_high_abs_feat if item not in columns_with_nan_std]
    top_5_low_abs_feat = [item for item in top_5_low_abs_feat if item not in columns_with_nan_std]
    top_5_high_feat = [item for item in top_5_high_feat if item not in columns_with_nan_std]
    top_5_low_feat = [item for item in top_5_low_feat if item not in columns_with_nan_std]

    # Standard All Feature
    df_pv[main_feature_list] = scaler.fit_transform(df_pv[main_feature_list])

    # USE ONLY ONE FACTOR
    ## Canberra Distance
    if USE_ONE_FEATURE_C :
        for feat in main_feature_list :
            df_nn = df_pv[['time_id',feat]]
            df_nn.set_index('time_id', inplace=True)
            df_nn = df_nn.fillna(df_nn.mean())

            time_id_neighbors.append(
                TimeIdNeighbors(
                    feat + '_c', 
                    df_nn, 
                    p=2, 
                    metric='canberra', 
                    exclude_self=True
                )
            )
    ## Manhattan Distance
    
    if USE_ONE_FEATURE_M_1:
        for feat in main_feature_list :
            df_nn = df_pv[['time_id',feat]]
            df_nn.set_index('time_id', inplace=True)
            df_nn = df_nn.fillna(df_nn.mean())

            time_id_neighbors.append(
                TimeIdNeighbors(feat + '_m_p1', df_nn, p=1)
            )

    ## Euclidean Distance

    if USE_ONE_FEATURE_M_2:
        for feat in main_feature_list :
            df_nn = df_pv[['time_id',feat]]
            df_nn.set_index('time_id', inplace=True)
            df_nn = df_nn.fillna(df_nn.mean())

            time_id_neighbors.append(
                TimeIdNeighbors(feat + '_m_p2', df_nn, p=2)
            )

    # TWO FACTOR

    if USE_TWO_FEATURES:
        feature_list = ['time_id','realized_volatility','bidask_spread_0']
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())

        ## Canberra

        time_id_neighbors.append(
                TimeIdNeighbors(
                    feat + 'two_c', 
                    df_nn, 
                    p=2, 
                    metric='canberra', 
                    exclude_self=True
                )
            )
        ## Euclidean Distance
        time_id_neighbors.append(
            TimeIdNeighbors(
                'two_m', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )

    # USE SEVALRAL FACTOR
    if USE_SEVALRAL_FEATURES:
        ## High Related Feature 
        feature_list = ['time_id']
        feature_list += top_5_high_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())

        ### Euclidean Distance
        time_id_neighbors.append(
            TimeIdNeighbors(
                'sev_high_nn_m', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )

        ## Low Related Feature

        feature_list = ['time_id']
        feature_list += top_5_low_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())        

        time_id_neighbors.append(
            TimeIdNeighbors(
                'sev_low_nn_m', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )

        ## High Abs Related Feature

        feature_list = ['time_id']
        feature_list += top_5_high_abs_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())
        
        time_id_neighbors.append(
            TimeIdNeighbors(
                'sev_high_abs_nn_m', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )

        ## Low Abs Related Feature

        feature_list = ['time_id']
        feature_list += top_5_low_abs_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())

        time_id_neighbors.append(
            TimeIdNeighbors(
                'sev_low_abs_nn_m', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )


    # USE ALL FACTOR

    if USE_ALL_FEATURES:
        df_nn = df_pv.copy()
        df_nn = df_nn.drop(['dv1_realized_volatility'], axis=1)
        df_nn.set_index('time_id', inplace=True)
        df_nn = df_nn.fillna(df_nn.mean())

        time_id_neighbors.append(
            TimeIdNeighbors(
                'all_nn_m_p1', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )

        time_id_neighbors.append(
            TimeIdNeighbors(
                'all_nn_m_p2', 
                df_nn, 
                p=2, 
                exclude_self=True
            )
        )


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


KeyError: "['volume_power'] not in index"

In [62]:
columns_with_nan_std

Index(['volume_power_150', 'volume_power_300', 'volume_power_450',
       'liq_last_1', 'liq_last_2', 'liq_last_5', 'liq_last_10', 'liq_last_15',
       'liq_last_1_300', 'liq_last_2_300', 'liq_last_5_300', 'liq_last_10_300',
       'liq_last_15_300'],
      dtype='object')

In [65]:
feature_list

['time_id', 'highest_return', 'volume_power']

#### Aggregate Features With NN

In [ ]:
def make_nearest_neighbor_feature(df: pd.DataFrame) -> pd.DataFrame:
    df2 = combined_result_df.copy()
    print(df2.shape)

    ### time_id를 기준으로 얻어진 neighbor를 대상으로 feature 만들기
    feature_cols = {
        'realized_volatility': [np.mean, np.min, np.max, np.std],
        'lowest_return': [np.max, np.mean, np.min],
        'num_trades': [np.mean],
        'trade.tau': [np.mean],
        'trade_vol': [np.mean],
        'dv1_realized_volatility': [np.mean],
        'bidask_spread_1': [np.mean],
        'bidask_spread_0': [np.mean],
        'tvpl': [np.mean],
        'tvpl_epliq5': [np.mean],
        'high_low_gap': [np.mean],
        'BB_width_w10': [np.mean],
        'BB_width_w20': [np.mean],
        'high_low_gap': [np.mean],
       
    }

    time_id_neigbor_sizes = [2, 4, 8, 16, 32, 48, 64]

    ndf: Optional[pd.DataFrame] = None
    
    # 새로운 feature를 기존 df에 추가하는 함수
    def _add_ndf(ndf: Optional[pd.DataFrame], dst: pd.DataFrame) -> pd.DataFrame:
        if ndf is None:
            return dst
        else:
            ndf[dst.columns[-1]] = dst[dst.columns[-1]].astype(np.float32)
            #columns_to_convert = [dst.columns[-1]]  # 열 변환 대상을 선택하거나 여러 열을 지정할 수 있음
            #converted_columns = dst[columns_to_convert].astype(np.float32)
            #ndf = pd.concat([ndf, converted_columns], axis=1)

            return ndf

    # neighbor time_id
    for feature_col in feature_cols.keys():
        try: 
            for nn in time_id_neighbors:
                nn.rearrange_feature_values(df2, feature_col)


            time_id_ns = time_id_neigbor_sizes

            for agg in feature_cols[feature_col]:
                for n in time_id_ns:
                    try:
                        for nn in time_id_neighbors:
                            dst = nn.make_nn_feature(n, agg)
                            ndf = _add_ndf(ndf, dst)
                    except Exception:
                        WHERE_ERROR = feature_col
                        print_trace('time-id nn')
                        pass
        except Exception:
            print_trace('time-id nn')

    if ndf is not None:
        df2 = pd.merge(df2, ndf, on=['time_id'], how='left')
    
    print(df2.shape)

    return df2

In [ ]:
gc.collect()

with timer('make nearest neighbor feature'):
    df3 = make_nearest_neighbor_feature(combined_result_df)

gc.collect()

NameError: name 'gc' is not defined

In [ ]:
df3.to_csv(working_directory + "output\\{}_sum_plus_nn_features.csv".format(coin), index=False)

NameError: name 'df3' is not defined

## NN PREDICTION
- NN을 기준으로 예측한 dv1_rv 값
- NN을 기준으로 비슷한 경우들의 dv1_rv 값들의 평균 (비슷했던 경우들의 평균)

아래 과정은 PASS 가능

In [ ]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [ ]:
target_value = "dv1_realized_volatility"
predict_list = []
for item in df3.columns:
    if target_value in item:
        predict_list.append(item)

predict_list = predict_list[1:]


predict = {}
for item in predict_list :
    predict[item] = rmspe(
        np.array(df3["dv1_realized_volatility"]),
        np.array(df3[item])
    )

In [ ]:
sorted_items = sorted(predict.items(), key=lambda x: x[1])

for key, value in sorted_items[:10]:
    print(key, value)

In [ ]:
# Save DF
# df3.to_parquet("my.parquet")

### FEATURE SELECTION
